# Tutorial

A single simulation run in Theatre_Ag is called an episode.  An episode needs a cast of actors and some directions, all synchronized on a clock.  This tutorial describes the steps for creating an episode.

All features of Theatre are accessible from the top level package.

In [2]:
from theatre_ag import Cast, Episode, SynchronizingClock, TaskQueueActor, default_cost

## Creating a Simulation Clock

In [3]:
clock = SynchronizingClock(max_ticks=5)

## Implementing the Cast

The abstract Actor class is responsible for executing tasks, synchronized on the Scenario clock.  However, it doesn't
have a mechanism for deciding how to schedule tasks, as this is left for implementation in the problem domain.  A simple
task queue oriented actor is provided with Theatre_Ag:

In [4]:
cast = Cast()
actor = TaskQueueActor('alice', clock)
cast.add_member(actor)

## Directing the Cast with Workflows

A workflow is represented as a plain old Python class, comprising attributes (which can be used to store references to
state in the simulation domain) and methods for manipulating the state.  For example, a class to represent the possible
states of the cleanliness of your hands can be given as:

In [5]:
class Hands(object):
    def __init__(self):
        self.clean = False
        self.soaped = False

The workflow for washing your hands (or anything else that can be washed) is defined as a plain old Python class, with
individual tasks represented as instance methods.

In [7]:
class WashWorkflow(object):

    is_workflow = True

    def __init__(self, washable):
        self.washable = washable

    @default_cost(1)
    def add_soap(self):
        self.washable.soaped = True

    @default_cost(1)
    def rinse(self):
        self.washable.soaped = False

    @default_cost(1)
    def scrub(self):
        if self.washable.soaped:
            self.washable.clean = True

    def wash(self):
        self.add_soap()
        self.scrub()
        self.rinse()

Task methods can contain the full range of Python control structures, including while and for loops, if-elif-else blocks
and <code>try-catch-finally</code> blocks.  Task methods can also freely invoke other task methods, and can pass
arguments and accept return values as desired.

Notice that tasks can be annotated with an indicative <code>default_cost</code>.  These annotations tell Theatre_Ag how
many clock ticks must be issued before the task is executed.  Generally, it is recommended to annotate methods with
their 'fixed' costs.  If a method contains a loop then it is better to encapsulate any activity in the loop that can
incur cost in a separate sub-task.

An instance of this workflow can be directly allocated to a TaskQueueActor.  The actor's <code>allocate_task</code>
method accepts a workflow entry point (a reference to a workflow instance method).

In [8]:
hands = Hands()
wash_workflow = WashWorkflow(hands)
actor.allocate_task(wash_workflow.wash)

wash()[?->?]

However, if you are dealing with an episode involving a large number of actors, it may be more convenient to specify
directions to a cast for the episode:

In [9]:
class WashHandsDirection(object):

    def apply(self, cast):
        hands = Hands()
        wash_workflow = WashWorkflow(hands)
        cast[0].allocate_task(wash_workflow.wash)

Workflows can be hierarchical, so it is possible to define (in the <code>__init__</code> method) sub workflows that can
be invoked from the parent.  For example, suppose we decide that rinsing is a sub-workflow that might be used in several
different workflows (perhaps a cleaning workflow that doesn't involve soap). Then we can separate out this part of the
overall workflow like this:

In [10]:
class RinseWorkflow(object):

    is_workflow = True

    def __init__(self, washable):
        self.washable = washable

    @default_cost(1)
    def rinse(self):
        self.washable.soaped = False


class WashWorkflow(object):

    is_workflow = True

    def __init__(self, washable):
        self.washable = washable
        self.rinse = RinseWorkflow(washable)

    @default_cost(1)
    def add_soap(self):
        self.washable.soaped = True

    @default_cost(1)
    def scrub(self):
        if self.washable.soaped:
            self.washable.clean = True

    @default_cost(1)
    def wash(self):
       self.add_soap()
       self.scrub()
       self.rinse.rinse()

## Playing an Episode and Gathering Results

Putting all this together, we can now define an episode of the problem domain and execute it as follows.

In [11]:
direction = WashHandsDirection()
episode = Episode(clock, cast, direction)
episode.perform()

Data for analysis can be collected from the problem domain, since these are just plain old Python objects.  For example:

In [12]:
print hands.clean

True


It is also possible to gather executed task history information from an Actor.  This information is stored in a
hierarchical tree of executed tasks.  Each task maintains a reference to its parent and child sub-tasks, if any.
Task objects also store the workflow, entry point, start and end times.  In the example episode above, the actor will
have exactly two wash tasks in its history.

In [13]:
print actor.task_history

[wash()[0->3], wash()[3->?]]


The first wash task will have one add_soap rinse sub-task, one scrub task, each of which consumed one clock tick.

In [14]:
print actor.task_history[0].sub_tasks

[add_soap()[0->1], scrub()[1->2], rinse()[2->3]]


The second sub-task will terminate after the scrub sub-task, as the clock reaches it's maximum tick:

In [15]:
print actor.task_history[1].sub_tasks

[add_soap()[4->5], scrub()[5->5]]
